In [4]:
import networkx as nx
import pandas as pd


In [5]:
def reformatdict(data):
    allPairs=[]
    for key in data.keys():
        for i in range(len(data[key])-1):
            allPairs.append((data[key][i],data[key][i+1]))
    #get rid of duplites
    allPairs = list(set(allPairs))
    
    
    allPairs2 =[]
    for x in allPairs:
        allPairs2.append(list(x))
    #allPairs.append(getGoodFormat( doug[1][key]))
    return allPairs2

In [18]:

edges = pd.read_csv('./data/mergedData/averagedTripDuration.csv')
nodes = pd.read_csv("./data/mergedData/nodes.csv")

nodes = nodes.drop('Unnamed: 0', axis =1)
edges = edges.drop('Unnamed: 0', axis =1)
edges = edges.rename(columns = {'duration': 'value'})

In [21]:
nodes.head()

,lat,long,id
0,40.657916,-73.925333,bu1000
1,40.659266,-73.927177,bu1001
2,40.660909,-73.929008,bu1002
3,40.663266,-73.931711,bu1003
4,40.664516,-73.937023,bu1004


In [22]:

nodesIds = nodes['id']
G=nx.Graph()

#add nodes
G.add_nodes_from(nodes)

#add edges
for i, row in edges.iterrows():
    G.add_edge(row['source'],row['target'],weight=row['value'])

#start with random bus stop to all other stops
nodeEdgesFrombibu4 =  nx.single_source_dijkstra(G,'bu1004')
nodeEdges = nodeEdgesFrombibu4[1]
nodeDistances =nodeEdgesFrombibu4[0]

nodeEdges = reformatdict(nodeEdges)
nodeEdges = pd.DataFrame(nodeEdges)
nodeEdges.rename(columns = {0:'source',1:'target'},inplace=True)
#nodeEdges['value'] = pd.Series(nodeDistances)

seriesNodes = pd.Series(nodeDistances)

vals = list(seriesNodes.values)
keys= list(seriesNodes.index)
newSeriesNodes= pd.DataFrame([vals,keys]).T
newSeriesNodes.rename(columns = {0:'source',1:'target'},inplace=True)


nodeEdges = nodeEdges.merge(newSeriesNodes, how = 'inner', left_on = 'target', right_on = 'target')
nodeEdges.rename(columns = {'source_x' : 'source', 'source_y':'value'},inplace =True)


In [24]:
allPairsDistances = nx.all_pairs_shortest_path_length(G)  #get optimal path lengths for every pair of locations
allPairsDistancesBig =  pd.DataFrame(allPairsDistances)

cols= list(allPairsDistancesBig.columns.values)
allPairsDistancesBig =allPairsDistancesBig.reset_index()
allPairsGoodFormat = pd.melt(allPairsDistancesBig, id_vars = ['index'], value_vars=cols)

allPairsGoodFormat.rename(columns = {'index':'source','variable':'target'},inplace =True)
allPairsGoodFormat.to_csv('./data/mergedData/allPairDistances.csv')

In [23]:
nodeEdges.head()

,source,target,value
0,butr427,bu1133,17.9435
1,bu3989,bu3992,55.2632
2,bu2378,bu2379,54.7538
3,butr69,butr68,13.3741
4,bu2107,bu2108,26.5095


In [ ]:
nodeEdges.to_csv('./data/mergedData/allPathsFrom.csv')
